# Session 11: Advanced Retrieval with LangChain

## Learning Objectives:

- Understand and implement multiple retrieval strategies for RAG
- Compare naive, BM25, multi-query, parent-document, contextual compression, ensemble, and semantic chunking approaches
- Build RAG chains over a health and wellness knowledge base using LangChain and QDrant

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

---

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

We'll also provide our OpenAI key, as well as our Cohere API key.

> NOTE: Create a `.env` file in this directory with `OPENAI_API_KEY` and `COHERE_API_KEY` to avoid being prompted each time.

In [1]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [2]:
if not os.environ.get("COHERE_API_KEY"):
    os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using our Health and Wellness Guide - a comprehensive resource covering exercise, nutrition, sleep, stress management, habits, and common health concerns.

### Data Preparation

We'll load the wellness guide as a single document, then split it into smaller chunks using a `RecursiveCharacterTextSplitter` for our vector store. We also keep the raw (unsplit) document for use with the Parent Document Retriever and Semantic Chunker later.

In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = TextLoader("data/HealthWellnessGuide.txt")
raw_docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
wellness_docs = text_splitter.split_documents(raw_docs)

Let's verify our data was loaded and split correctly!

In [4]:
print(f"Raw documents: {len(raw_docs)}")
print(f"Split chunks: {len(wellness_docs)}")
print(f"\nExample chunk:\n{wellness_docs[0]}")

Raw documents: 1
Split chunks: 45

Example chunk:
page_content='The Personal Wellness Guide
A Comprehensive Resource for Health and Well-being

PART 1: EXERCISE AND MOVEMENT

Chapter 1: Understanding Exercise Basics

Exercise is one of the most important things you can do for your health. Regular physical activity can improve your brain health, help manage weight, reduce the risk of disease, strengthen bones and muscles, and improve your ability to do everyday activities.' metadata={'source': 'data/HealthWellnessGuide.txt'}


## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "wellness_guide".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [5]:
from langchain_qdrant import QdrantVectorStore
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = QdrantVectorStore.from_documents(
    wellness_docs,
    embeddings,
    location=":memory:",
    collection_name="wellness_guide",
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [6]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [7]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-4.1-nano` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [8]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4.1-nano")

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [9]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [10]:
naive_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

"Exercises that can help with lower back pain include:\n\n- Cat-Cow Stretch: Start on hands and knees, alternate between arching your back up (cat) and letting it sag down (cow). Perform 10-15 repetitions.\n\n- Bird Dog: From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold each extension for about 5 seconds, then switch sides. Do 10 repetitions per side.\n\n- Pelvic Tilts: Lie on your back with knees bent, flatten your back against the floor by tightening your abs and tilting your pelvis up slightly. Hold for 10 seconds and repeat 8-12 times.\n\n- Partial Crunches: Lie on your back with knees bent, cross arms over your chest, tighten your stomach muscles, and lift your shoulders off the floor briefly before lowering. Aim for 8-12 repetitions.\n\n- Knee-to-Chest Stretch: Lie on your back, pull one knee toward your chest while keeping the other foot flat on the ground. Hold for 15-30 seconds, then switch legs.\n\nThese exercises are gentle and can help 

In [15]:
naive_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a vital role in maintaining overall health. It is essential for physical recovery, as during sleep the body repairs tissues and regenerates. Sleep also supports mental well-being and cognitive functions, including memory consolidation and learning. Adequate sleep (7-9 hours per night) helps regulate hormones related to growth and appetite, boosts immune function, and improves emotional and mental health. Poor sleep or sleep disorders like insomnia can negatively impact health by impairing these processes, increasing the risk of conditions such as anxiety, depression, weakened immunity, and other health issues. Therefore, maintaining good sleep hygiene and a regular sleep routine is crucial for overall health and well-being.'

In [19]:
naive_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- Drinking water and staying hydrated\n- Applying cold or warm compresses to the head or neck\n- Resting in a dark, quiet room\n- Gentle massage of the temples and neck\n- Using peppermint or lavender essential oils\n- Maintaining a regular sleep schedule\n- Practicing deep breathing, progressive muscle relaxation, or grounding techniques\n- Taking short walks in nature\n- Listening to calming music\n\nThese approaches can help alleviate stress and headache symptoms naturally.'

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [10]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(wellness_docs)

We'll construct the same chain - only changing the retriever.

In [11]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [13]:
bm25_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'To help with lower back pain, some recommended exercises include:\n\n- **Cat-Cow Stretch:** Start on your hands and knees, then alternate between arching your back upward (cat) and letting it sag downward (cow). Repeat 10-15 times.\n- **Bird Dog:** From your hands and knees, extend opposite arm and leg while keeping your core engaged. Hold each extension for 5 seconds and switch sides. Do 10 repetitions per side.\n- **Pelvic Tilts:** Lie on your back with knees bent, tighten your abdominal muscles, and tilt your pelvis slightly upward to flatten your back against the floor. Hold for 10 seconds and repeat 8-12 times.\n\nThese gentle stretching and strengthening exercises can help alleviate discomfort and prevent future episodes of lower back pain.'

In [14]:
bm25_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a crucial role in overall health. It occurs in cycles of about 90 minutes, including REM and non-REM stages, each serving important functions. Deep sleep (Stage 3) is when the body repairs and regenerates tissues, supporting physical health. REM sleep is vital for brain activity, memory, and learning. \n\nGetting enough quality sleep—typically 7-9 hours for adults—and creating an optimal sleep environment with appropriate temperature, darkness, quiet, and comfortable bedding can improve health. Sleep helps regulate immune function, mental health, and physical recovery, making it a foundational component of overall wellness.'

In [24]:
bm25_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress include practicing relaxation techniques such as deep breathing and meditation. For headaches, staying well-hydrated, managing stress through relaxation, ensuring adequate sleep, and avoiding known triggers like certain foods or eye strain can help reduce their frequency and intensity. Herbal teas like chamomile or valerian root may also promote relaxation and help alleviate headaches related to stress.'

It's not clear that this is better or worse, if only we had a way to test this (SPOILERS: We do, the second half of the notebook will cover this)

### ❓ Question #1:

Give an example query where BM25 is better than embeddings and justify your answer.

##### Answer:
In this small and well-structured dataset, both BM25 and embeddings return identical correct answers for most queries. Because each concept appears in a single, clearly defined section, there is little ambiguity.

Therefore, it is difficult to construct a realistic query where BM25 significantly outperforms embeddings in retrieval quality. The main observable difference in this case is performance speed rather than answer correctness.

In larger or noisier datasets with repeated semantic content, BM25 would likely perform better for queries requiring exact lexical or numeric matches.

### Example 1

In [20]:
naive_retrieval_chain.invoke({"question" : "How many days per week must insomnia occur to be considered chronic?"})["response"].content

'Insomnia must occur at least 3 nights per week for 3 months or more to be considered chronic.'

In [26]:
bm25_retrieval_chain.invoke({"question" : "How many days per week must insomnia occur to be considered chronic?"})["response"].content

'Insomnia is considered chronic if it occurs at least 3 nights per week for 3 months or more.'

### Example 2

In [22]:
naive_retrieval_chain.invoke({"question" : "How long does it take on average to form a new habit?"})["response"].content

'On average, it takes about 66 days to form a new habit.'

In [23]:
bm25_retrieval_chain.invoke({"question" : "How long does it take on average to form a new habit?"})["response"].content

'Based on the information provided, it takes an average of 66 days to form a new habit.'

### Example 3

In [43]:
naive_retrieval_chain.invoke({"question" : "Where should I start slowly?"})["response"].content

"You should start slowly when building new exercise or wellness habits. For example, begin with just 2 minutes of a new habit, such as meditation or light movement, and gradually increase over time. When starting a workout routine, it's recommended to begin with shorter, manageable sessions like a 20-minute walk or 15 minutes of bodyweight exercises, and then gradually increase the duration and intensity. The key is to start small and be patient as habits and routines develop."

In [42]:
bm25_retrieval_chain.invoke({"question" : "Where should I start slowly?"})["response"].content

'Based on the information provided, you should start slowly by beginning with small, manageable steps. For example, you could start with just 2 minutes of activity, such as a brief walk or some gentle stretches, and gradually increase the duration and intensity over time. This approach helps prevent feeling overwhelmed and builds a sustainable habit.'

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [33]:
os.environ['REQUESTS_CA_BUNDLE'] = 'C:/Users/dznidaric/Documents/certs/cacert.pem'
os.environ['SSL_CERT_FILE'] = 'C:/Users/dznidaric/Documents/certs/cacert.pem'

In [40]:
import httpx

_original_client_init = httpx.Client.__init__

def _no_ssl_client_init(self, *args, **kwargs):
    kwargs.setdefault("verify", False)
    return _original_client_init(self, *args, **kwargs)

httpx.Client.__init__ = _no_ssl_client_init

In [41]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-v3.5")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [42]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [43]:
contextual_compression_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch:** Start on your hands and knees, then alternate between arching your back upward (cat) and sagging it downward (cow). Aim for 10-15 repetitions.\n- **Bird Dog:** From the same position on hands and knees, extend opposite arm and leg while keeping your core engaged. Hold each extension for about 5 seconds and switch sides. Do 10 repetitions per side.\n- **Pelvic Tilts:** Lie on your back with knees bent, flatten your back against the floor by tightening your abdominal muscles and tilting your pelvis slightly upward. Hold for 10 seconds and repeat 8-12 times.\n\nThese gentle stretching and strengthening exercises can help alleviate and prevent lower back pain.'

In [29]:
contextual_compression_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep has a significant impact on overall health. It is essential for physical recovery, as the body repairs tissues and regenerates during deep sleep stages. Sleep also plays a crucial role in mental well-being, helping with memory consolidation and cognitive functions. Additionally, sleep influences the release of hormones that regulate growth and appetite, which can affect overall metabolic health. Adults generally need 7-9 hours of quality sleep per night to support these vital functions and maintain good health.'

In [30]:
contextual_compression_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include drinking water to stay hydrated, applying cold or warm compresses to the head or neck, resting in a dark, quiet room, gently massaging the temples and neck, and using essential oils like peppermint or lavender. Additionally, practices such as deep breathing, progressive muscle relaxation, grounding techniques, taking short walks in nature, and listening to calming music can help alleviate stress.'

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [44]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
) 

In [45]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [46]:
multi_query_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- Cat-Cow Stretch: Start on your hands and knees, then alternate between arching your back up (cat) and letting it sag down (cow). Perform 10-15 repetitions.\n\n- Bird Dog: From hands and knees, extend the opposite arm and leg while keeping your core engaged. Hold each extension for about 5 seconds and switch sides. Do this for 10 repetitions on each side.\n\n- Partial Crunches: Lie on your back with knees bent, cross arms over your chest, tighten your stomach muscles, and lift your shoulders off the floor briefly before lowering back down. Aim for 8-12 repetitions.\n\n- Knee-to-Chest Stretch: Lie on your back, pull one knee toward your chest while keeping the other foot flat on the ground. Hold for 15-30 seconds and switch legs.\n\n- Pelvic Tilts: Lie on your back with knees bent, flatten your back against the floor by tightening your abs and tilting your pelvis slightly upward. Hold for 10 seconds and repeat 8-12 times.\n\nThe

In [34]:
multi_query_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a vital role in overall health by supporting physical recovery, mental well-being, and cognitive functions. During sleep, the body repairs tissues, consolidates memories, and releases hormones that help regulate growth and appetite. Adequate and quality sleep—typically 7-9 hours per night—also contributes to a stronger immune system, better emotional regulation, and overall longevity. Poor sleep or sleep disturbances, such as insomnia, can negatively impact physical health, increase stress, and reduce mental clarity. Therefore, maintaining good sleep hygiene and creating an optimal sleep environment are essential for promoting overall health and well-being.'

In [35]:
multi_query_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- Deep breathing exercises (inhale for 4 counts, hold for 4, exhale for 4)\n- Progressive muscle relaxation (tensing and releasing muscle groups)\n- Grounding techniques (naming things you see, hear, feel, smell, and taste)\n- Taking short walks, preferably in nature\n- Listening to calming music\n- Drinking water to stay hydrated\n- Applying cold or warm compresses to the head or neck\n- Resting in a dark, quiet room\n- Gentle massage of temples and neck\n- Using essential oils like peppermint or lavender\n\nThese methods can help provide immediate relief and promote overall relaxation.'

### ❓ Question #2:

Explain how generating multiple reformulations of a user query can improve recall.

##### Answer:

Generating multiple reformulations of a user query improves recall by reducing vocabulary mismatch.
Different documents may use different words or phrasing to describe the same concept. If retrieval relies on a single query, relevant documents that use alternative wording may be missed.

By creating several variations of the query, the retriever searches using multiple phrasings, increasing the likelihood of matching relevant documents. In LangChain, *MultiQueryRetriever* uses an LLM to generate these alternative queries and combines the results, which improves overall recall.

Even in embedding-based retrieval systems, query reformulation can help. Although embeddings capture semantic similarity, they may still miss relevant documents due to wrong wording, ambiguity, or limitations in vector representation. Multiple reformulations create multiple semantic search vectors, expanding the search space.

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. We split the full document into large "parent" chunks (e.g. 2000 characters).
2. Each parent chunk is further split into smaller "child" chunks (e.g. 400 characters).
3. The child chunks are stored in a VectorStore, while the parent chunks are stored in an in-memory docstore.
4. When we query our Retriever, we do a similarity search comparing our query vector to the child chunks.
5. Instead of returning the child chunks, we return their associated parent chunks.

The basic idea is:

- **Search** for small, focused chunks (better semantic matching)
- **Return** big chunks (richer surrounding context)

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by defining our parent and child splitters.

In [47]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [48]:
from langchain_qdrant import QdrantVectorStore

client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="wellness_parent_child",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = QdrantVectorStore(
    collection_name="wellness_parent_child", embedding=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [49]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore=parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [50]:
parent_document_retriever.add_documents(raw_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [51]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [52]:
parent_document_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'For lower back pain, the following exercises can help alleviate discomfort and prevent future issues:\n\n- **Cat-Cow Stretch:** Start on your hands and knees. Alternate between arching your back up (Cat) and letting it sag down (Cow). Perform 10-15 repetitions.\n- **Bird Dog:** From hands and knees, extend opposite arm and leg while keeping your core engaged. Hold each side for 5 seconds. Do 10 repetitions per side.\n- **Partial Crunches:** Lie on your back with knees bent, arms crossed over your chest. Tighten your stomach muscles and raise your shoulders off the floor briefly, then lower. Do 8-12 repetitions.\n- **Knee-to-Chest Stretch:** Lie on your back and pull one knee toward your chest while keeping the other foot flat on the ground. Hold for 15-30 seconds, then switch legs.\n- **Pelvic Tilts:** Lie on your back with knees bent. Flatten your back against the floor by tightening your abs and tilting your pelvis up slightly. Hold for 10 seconds. Repeat 8-12 times.\n\nThese gentle

In [42]:
parent_document_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep significantly affects overall health in several ways. It is crucial for physical health, mental well-being, and cognitive function. During sleep, the body repairs tissues, consolidates memories, and releases hormones that regulate growth and appetite. Adults generally need 7-9 hours of sleep per night. Good sleep quality supports healthy immune function, maintains mood stability, and enhances concentration and learning. Poor sleep or sleep disturbances can lead to increased risk of health issues such as fatigue, headaches, impaired cognitive function, and emotional stress. Therefore, maintaining healthy sleep habits and hygiene is vital for overall wellness.'

In [43]:
parent_document_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include practicing deep breathing, engaging in progressive muscle relaxation, doing grounding exercises, taking short walks especially in nature, listening to calming music, and practicing mindfulness or meditation. For headaches specifically, natural remedies include drinking plenty of water to stay hydrated, applying cold or warm compresses to the head or neck, resting in a dark, quiet room, gently massaging the temples and neck, using essential oils like peppermint or lavender, and maintaining a regular sleep schedule.'

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [53]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [54]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [46]:
ensemble_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch:** Start on hands and knees, alternate between arching your back up (like a cat) and letting it sag down (like a cow). Do 10-15 repetitions.\n\n- **Bird Dog:** From hands and knees, extend the opposite arm and leg while keeping your core engaged. Hold for 5 seconds, then switch sides. Do 10 repetitions per side.\n\n- **Pelvic Tilts:** Lie on your back with knees bent, tighten your abs, and tilt your pelvis slightly to flatten your lower back against the floor. Hold for 10 seconds, repeat 8-12 times.\n\n- **Partial Crunches:** Lie on your back with knees bent, cross your arms over your chest, tighten your stomach muscles, and lift your shoulders off the floor briefly. Then lower back down. Do 8-12 repetitions.\n\n- **Knee-to-Chest Stretch:** Lie on your back, pull one knee toward your chest while keeping the other foot flat on the floor. Hold for 15-30 seconds, then switch legs.\n\nRemember to start slowly and

In [55]:
ensemble_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep plays a vital role in overall health. It is essential for physical recovery, mental well-being, and cognitive function. During sleep, the body repairs tissues, consolidates memories, and releases hormones that regulate growth and appetite. Getting adequate sleep—typically 7 to 9 hours per night—supports a strong immune system, helps manage stress, and maintains emotional stability. Poor sleep or sleep disorders like insomnia can impair these processes, leading to health issues such as weakened immunity, increased stress, impaired concentration, and higher risk for chronic conditions. Therefore, maintaining good sleep hygiene and creating an optimal sleep environment are crucial strategies for promoting overall health and wellness.'

In [48]:
ensemble_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress include deep breathing exercises, progressive muscle relaxation, grounding techniques (such as naming things you see, hear, feel, smell, and taste), taking short walks especially in nature, and listening to calming music. \n\nFor headaches, natural remedies include staying well-hydrated by drinking water, applying cold or warm compresses to the head or neck, resting in a dark and quiet room, gentle massage of the temples and neck, using essential oils such as peppermint or lavender, maintaining a regular sleep schedule, and using caffeine in small amounts if appropriate.'

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [56]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [57]:
semantic_documents = semantic_chunker.split_documents(raw_docs)

Let's create a new vector store.

In [58]:
semantic_vectorstore = QdrantVectorStore.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="wellness_guide_semantic_chunks"
)

We'll use naive retrieval for this example.

In [59]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

Finally we can create our classic chain!

In [60]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [55]:
semantic_retrieval_chain.invoke({"question" : "What exercises can help with lower back pain?"})["response"].content

'Exercises that can help with lower back pain include:\n\n- **Cat-Cow Stretch:** Start on hands and knees. Alternate between arching your back up (cat position) and letting it sag down (cow position). Do 10-15 repetitions.\n\n- **Partial Crunches:** Lie on your back with knees bent, cross arms over chest, tighten stomach muscles, and raise shoulders off the floor. Hold briefly, then lower. Do 8-12 repetitions.\n\n- **Knee-to-Chest Stretch:** Lie on your back, pull one knee toward your chest while keeping the other foot flat. Hold for 15-30 seconds, then switch legs.\n\n- **Pelvic Tilts:** Lie on your back with knees bent, flatten your back against the floor by tightening your abs and tilting your pelvis up slightly. Hold for 10 seconds and repeat 8-12 times.\n\nThese exercises, when done gently and regularly, can help alleviate discomfort and prevent future episodes of lower back pain. However, it’s always best to consult with a healthcare professional before starting any new exercise 

In [56]:
semantic_retrieval_chain.invoke({"question" : "How does sleep affect overall health?"})["response"].content

'Sleep is essential for overall health because it allows the body to repair tissues, regulate hormones, and support cognitive functions like memory and learning. Adequate sleep (7-9 hours per night) helps maintain physical health, mental well-being, and proper immune function. Poor sleep or disruptions in sleep cycles can lead to issues such as fatigue, headaches, weakened immune response, and impaired mental health. Good sleep hygiene practices and creating an optimal sleep environment are important for ensuring restful sleep and promoting overall wellness.'

In [61]:
semantic_retrieval_chain.invoke({"question" : "What are some natural remedies for stress and headaches?"})["response"].content

'Some natural remedies for stress and headaches include:\n\n- For stress:\n  - Deep breathing exercises, such as inhaling for 4 counts, holding for 4, exhaling for 4, and holding again\n  - Progressive muscle relaxation, tensing and releasing muscle groups\n  - Grounding techniques, like listing items you can see, hear, feel, smell, and taste\n  - Taking short walks in nature\n  - Listening to calming music\n  - Practicing mindfulness and meditation regularly, such as focused attention, body scan, loving-kindness, or walking meditation\n\n- For headaches:\n  - Drinking plenty of water to stay hydrated\n  - Applying cold or warm compresses to the head or neck\n  - Resting in a quiet, dark room\n  - Gentle massage of temples and neck\n  - Using essential oils like peppermint or lavender\n  - Maintaining regular sleep schedules\n  - Managing stress through relaxation techniques\n\nThese approaches emphasize relaxation, hydration, and mindfulness, which can help alleviate stress and headac

### ❓ Question #3:

If sentences are short and highly repetitive (e.g., FAQs), how might semantic chunking behave, and how would you adjust the algorithm?

##### Answer:

If sentences are short and highly repetitive, semantic chunking may behave poorly because many adjacent sentences will have very similar embeddings. This can cause the algorithm to merge too many Q/A items into large chunks, reducing retrieval precision (the retriever then returns a big chunk with multiple unrelated FAQs).

To adjust, I would:
- Lower the chunk-size ceiling (max sentences/tokens per chunk), based on the average faq size
- Add structural split rules (eg., split on “Q:” / “A:” markers and/or blank lines) before applying semantic similarity
- Use a stricter threshold or different methods, like gradient, to detect real topic shifts rather than small similarity changes
- Embed Q/A pairs as atomic units (treat each FAQ entry as one document) instead of sentence-level chunking

---

# 🤝 Breakout Room Part #2

### 🏗️ Activity #1:

Your task is to evaluate the various Retriever methods against each other.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparison between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

In [ ]:
import os
from uuid import uuid4

import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

if not os.environ.get("COHERE_API_KEY"):
    os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = os.environ.get("LANGCHAIN_API_KEY")
os.environ["LANGSMITH_ENDPOINT"] = "https://eu.api.smith.langchain.com"

if not os.environ["LANGCHAIN_API_KEY"]:
    os.environ["LANGSMITH_TRACING"] = "false"
    print("LangSmith tracing disabled")
else:
    print(f"LangSmith tracing enabled. Project: {os.environ['LANGCHAIN_PROJECT']}")

LangSmith tracing enabled. Project: AIM - SDG - f20bce8f


In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/HealthWellnessGuide.txt")
raw_docs = loader.load()

In [64]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

c:\Users\dznidaric\Documents\github\AIE9\11_Advanced_Retrieval\.venv\Lib\site-packages\pysbd\segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
c:\Users\dznidaric\Documents\github\AIE9\11_Advanced_Retrieval\.venv\Lib\site-packages\pysbd\lang\arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
c:\Users\dznidaric\Documents\github\AIE9\11_Advanced_Retrieval\.venv\Lib\site-packages\pysbd\lang\persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


In [ ]:
from ragas.testset.graph import KnowledgeGraph
from ragas.testset.graph import Node, NodeType
from ragas.testset.transforms import default_transforms, apply_transforms

kg = KnowledgeGraph()

for doc in raw_docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )


transforms = default_transforms(documents=raw_docs, llm=generator_llm, embedding_model=generator_embeddings)
apply_transforms(kg, transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/1 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/4 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/9 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 5, relationships: 8)

In [74]:
kg.save("usecase_data_kg.json")
usecase_data_kg = KnowledgeGraph.load("usecase_data_kg.json")
usecase_data_kg

KnowledgeGraph(nodes: 5, relationships: 8)

In [ ]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(
    llm=generator_llm,
    embedding_model=generator_embeddings,
    knowledge_graph=usecase_data_kg,
)
dataset = generator.generate_with_langchain_docs(raw_docs, testset_size=10)

dataset.to_pandas()

Applying HeadlinesExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/1 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/1 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/3 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/7 [00:00<?, ?it/s]

...

Generating Scenarios:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What are macronutrients and why are they impor...,[PART 2: NUTRITION AND DIET Chapter 4: Fundame...,Macronutrients are the key components of a bal...,single_hop_specifc_query_synthesizer
1,What role do vitamins play in a balanced diet?,[PART 2: NUTRITION AND DIET Chapter 4: Fundame...,Vitamins are organic compounds needed in small...,single_hop_specifc_query_synthesizer
2,What are Proteins and why are they important f...,[PART 2: NUTRITION AND DIET Chapter 4: Fundame...,Proteins are essential for muscle repair and i...,single_hop_specifc_query_synthesizer
3,How do carbohydrates function as a primary ene...,[PART 2: NUTRITION AND DIET Chapter 4: Fundame...,Carbohydrates are described as the primary ene...,single_hop_specifc_query_synthesizer
4,What does work-life balance mean?,[13: The Science of Habit Formation Habits are...,Work-life balance refers to maintaining a heal...,single_hop_specifc_query_synthesizer
5,How do social connections help improve health ...,[13: The Science of Habit Formation Habits are...,Building and maintaining social connections ar...,single_hop_specifc_query_synthesizer
6,What are natural remedies for headaches?,[13: The Science of Habit Formation Habits are...,Natural headache remedies include drinking wat...,single_hop_specifc_query_synthesizer
7,wht are morning routines,[13: The Science of Habit Formation Habits are...,"According to the context, morning routines are...",single_hop_specifc_query_synthesizer
8,How does Monday contribute to a balanced weekl...,[The Personal Wellness Guide A Comprehensive R...,"According to the Personal Wellness Guide, Mond...",single_hop_specifc_query_synthesizer
9,Exercise what good for health?,[The Personal Wellness Guide A Comprehensive R...,Exercise is one of the most important things y...,single_hop_specifc_query_synthesizer


In [78]:
from langsmith import Client
import uuid

client = Client()

dataset_name = f"Use Case Synthetic Data - AIE9 - {uuid.uuid4()}"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Synthetic Data for Use Cases"
)

for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_qdrant import QdrantVectorStore

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
wellness_docs = text_splitter.split_documents(raw_docs)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = QdrantVectorStore.from_documents(
    wellness_docs,
    embeddings,
    location=":memory:",
    collection_name="wellness_guide",
)

In [81]:
semantic_vectorstore = None
try:
    from langchain_experimental.text_splitter import SemanticChunker

    semantic_splitter = SemanticChunker(embeddings)
    semantic_docs = semantic_splitter.split_documents(raw_docs)

    semantic_vectorstore = QdrantVectorStore.from_documents(
        semantic_docs,
        embeddings,
        location=":memory:",
        collection_name="wellness_guide_semantic",
    )
    print("Semantic chunking index built.")
except Exception as e:
    print("Semantic chunking not available (langchain_experimental missing or incompatible). Skipping.")
    semantic_vectorstore = None

Semantic chunking index built.


In [83]:
# defined before

retrievers = {
    "bm25_retriever": bm25_retriever,
    "naive_retriever": naive_retriever,
    "parent_document_retriever": parent_document_retriever,
    "compression_retriever": compression_retriever,
    "multi_query_retriever": multi_query_retriever,
}

semantic_retrievers = None
if semantic_vectorstore is not None:
    semantic_naive = semantic_vectorstore.as_retriever(search_kwargs={"k": 4})
    semantic_retrievers = {
        "naive_retriever_semantic_chunks": semantic_naive,
    }

In [ ]:
import time
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import context_precision, context_recall

def run_retriever_eval(retriever, df):
    questions = []
    retrieved_contexts = []
    reference_contexts = []
    references = []
    latencies = []

    for _, row in df.iterrows():
        q = row["user_input"]
        ref = row["reference"]
        ref_ctx = row["reference_contexts"]

        t0 = time.perf_counter()
        docs = retriever.invoke(q)
        t1 = time.perf_counter()

        ctx = [d.page_content for d in docs]

        questions.append(q)
        references.append(ref)
        reference_contexts.append(ref_ctx)
        retrieved_contexts.append(ctx)
        latencies.append(t1 - t0)
        time.sleep(6.5)

    eval_ds = Dataset.from_dict({
        "question": questions,
        "contexts": retrieved_contexts,
        "reference_contexts": reference_contexts,
        "reference": references,
    })

    scores = evaluate(
        eval_ds,
        metrics=[context_precision, context_recall],
        # If your Ragas version requires mapping, keep this:
        column_map={
            "question": "question",
            "contexts": "contexts",
            "reference_contexts": "reference_contexts",
            "reference": "reference",
        },
    )

    return {
        "scores": scores,
        "avg_latency_s": sum(latencies) / max(1, len(latencies)),
        "latencies_s": latencies,
    }

In [ ]:
wellness_df = dataset.to_pandas()
wellness_df

,user_input,reference_contexts,reference,synthesizer_name
0,What are the macronutrients in a healthy diet?,[PART 2: NUTRITION AND DIET Chapter 4: Fundame...,"Macronutrients include carbohydrates, proteins...",single_hop_specifc_query_synthesizer
1,What role do proteins play in a healthy diet?,[PART 2: NUTRITION AND DIET Chapter 4: Fundame...,Proteins are essential for muscle repair and i...,single_hop_specifc_query_synthesizer
2,What are macronutrients?,[PART 2: NUTRITION AND DIET Chapter 4: Fundame...,Macronutrients are the key components of a bal...,single_hop_specifc_query_synthesizer
3,How do carbohydrates function as a primary ene...,[PART 2: NUTRITION AND DIET Chapter 4: Fundame...,Carbohydrates are described as the primary ene...,single_hop_specifc_query_synthesizer
4,What are natural remedies for headaches?,[13: The Science of Habit Formation Habits are...,Natural headache remedies include drinking wat...,single_hop_specifc_query_synthesizer
5,What does digital wellness entail?,[13: The Science of Habit Formation Habits are...,Digital wellness involves practices to manage ...,single_hop_specifc_query_synthesizer
6,Hwo can I imprve my digestve health to boost m...,[13: The Science of Habit Formation Habits are...,A healthy digestive system is foundational to ...,single_hop_specifc_query_synthesizer
7,What are some natural ways to support digestiv...,[13: The Science of Habit Formation Habits are...,Supporting digestive health involves eating fi...,single_hop_specifc_query_synthesizer
8,Wednesday what do I do,[The Personal Wellness Guide A Comprehensive R...,The guide suggests that Wednesday can be a res...,single_hop_specifc_query_synthesizer
9,What activities are recommended for Thursdays ...,[The Personal Wellness Guide A Comprehensive R...,"On Thursdays, the recommended activities inclu...",single_hop_specifc_query_synthesizer


In [94]:
import time
import json

def safe_eval_retriever(name, retriever, df, retries=3, backoff_s=5):
    for attempt in range(1, retries + 1):
        try:
            return run_retriever_eval(retriever, df)
        except json.JSONDecodeError as e:
            print(f"[{name}] JSONDecodeError attempt {attempt}/{retries}. Backing off {backoff_s}s...")
            time.sleep(backoff_s * attempt)
        except Exception as e:
            print(f"[{name}] Failed with {type(e).__name__}: {e}")
            raise  # for non-transient errors, fail fast
    raise RuntimeError(f"[{name}] failed after {retries} retries")

In [95]:
all_results = {}
for name, r in retrievers.items():
    out = safe_eval_retriever(name, r, wellness_df)
    all_results[name] = out
    print(f"\n=== {name} ===")
    print(out["scores"])
    print(f"Avg latency: {out['avg_latency_s']:.3f}s")

# Optional semantic chunk comparison
if semantic_retrievers:
    for name, r in semantic_retrievers.items():
        out = run_retriever_eval(r, wellness_df)
        all_results[name] = out
        print(f"\n=== {name} ===")
        print(out["scores"])
        print(f"Avg latency: {out['avg_latency_s']:.3f}s")

Evaluating:   0%|          | 0/24 [00:00<?, ?it/s]


=== bm25_retriever ===
{'context_precision': 0.1667, 'context_recall': 0.1806}
Avg latency: 0.002s


Evaluating:   0%|          | 0/24 [00:00<?, ?it/s]


=== naive_retriever ===
{'context_precision': 0.9005, 'context_recall': 0.9583}
Avg latency: 0.656s


Evaluating:   0%|          | 0/24 [00:00<?, ?it/s]

...


=== compression_retriever ===
{'context_precision': 0.8889, 'context_recall': 0.9722}
Avg latency: 0.623s


Evaluating:   0%|          | 0/24 [00:00<?, ?it/s]


=== multi_query_retriever ===
{'context_precision': 0.7708, 'context_recall': 0.9583}
Avg latency: 1.533s


Evaluating:   0%|          | 0/24 [00:00<?, ?it/s]


=== naive_retriever_semantic_chunks ===
{'context_precision': 0.8333, 'context_recall': 1.0000}
Avg latency: 0.310s


In [115]:
for name, out in all_results.items():
    s = out["scores"]
    print(name, type(s), s)
    print("precision type:", type(s["context_precision"]))
    print("recall type:", type(s["context_recall"]))

bm25_retriever <class 'ragas.dataset_schema.EvaluationResult'> {'context_precision': 0.1667, 'context_recall': 0.1806}
precision type: <class 'list'>
recall type: <class 'list'>
naive_retriever <class 'ragas.dataset_schema.EvaluationResult'> {'context_precision': 0.9005, 'context_recall': 0.9583}
precision type: <class 'list'>
recall type: <class 'list'>
parent_document_retriever <class 'ragas.dataset_schema.EvaluationResult'> {'context_precision': 0.9028, 'context_recall': 1.0000}
precision type: <class 'list'>
recall type: <class 'list'>
compression_retriever <class 'ragas.dataset_schema.EvaluationResult'> {'context_precision': 0.8889, 'context_recall': 0.9722}
precision type: <class 'list'>
recall type: <class 'list'>
multi_query_retriever <class 'ragas.dataset_schema.EvaluationResult'> {'context_precision': 0.7708, 'context_recall': 0.9583}
precision type: <class 'list'>
recall type: <class 'list'>
naive_retriever_semantic_chunks <class 'ragas.dataset_schema.EvaluationResult'> {'co

In [119]:
import pandas as pd
import numpy as np


def mean_or_nan(x):
    if isinstance(x, list):
        return float(np.mean(x)) if len(x) else float("nan")
    return float(x)


rows = []
for name, out in all_results.items():
    # scores is a Ragas EvaluationResult; convert to dict-like
    scores = out["scores"]
    cp = mean_or_nan(scores["context_precision"])
    cr = mean_or_nan(scores["context_recall"])
    rows.append(
        {
            "retriever": name,
            "context_precision": cp,
            "context_recall": cr,
            "avg_latency_s": out["avg_latency_s"],
        }
    )

results_df = pd.DataFrame(rows).sort_values(
    by=["context_recall", "context_precision"], ascending=False
)

display(results_df)

,retriever,context_precision,context_recall,avg_latency_s
2,parent_document_retriever,0.902778,1.000000,0.377952
5,naive_retriever_semantic_chunks,0.833333,1.000000,0.310293
3,compression_retriever,0.888889,0.972222,0.622531
1,naive_retriever,0.900463,0.958333,0.655711
4,multi_query_retriever,0.770833,0.958333,1.532813
0,bm25_retriever,0.166667,0.180556,0.001902
